In [1]:
import pickle
import pandas as pd
from transformers import TextClassificationPipeline, DistilBertForSequenceClassification, DistilBertTokenizer
import ast
import torch
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = pickle.load(open('embedding_model.pkl', 'rb'))
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
le = pickle.load(open('label_enc.pkl', 'rb'))

/opt/conda/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [2]:
def converter(text_string):
    new_list = []
    label_list = sorted(pipe(text_string)[0], key=lambda item: item["score"], reverse=True)[:5]
    for val in label_list:
        new_dict = {}
        new_dict[le.inverse_transform([int(val['label'].split('_')[1])])[0]] = round(val['score']*100,2)
        new_list.append(new_dict)
    return new_list

In [3]:
def run_pipeline(input_data, output_data):
    input_data = pd.read_csv(input_data).iloc[:,0].to_frame()
    input_data.columns = ['input_data']
    multiple_predictions = input_data.dropna()
    multiple_predictions['all_predictions'] = multiple_predictions['input_data'].apply(lambda x:converter(x))
    multiple_predictions = multiple_predictions.explode('all_predictions')
    multiple_predictions['predicted_category'] = multiple_predictions['all_predictions'].apply(lambda x:x.keys())
    multiple_predictions['prediction_probability'] = multiple_predictions['all_predictions'].apply(lambda x:x.values())
    multiple_predictions['model_predicted_category'] = multiple_predictions['predicted_category'].apply(lambda x: str(x).replace("dict_keys(['","").replace("'])",""))
    multiple_predictions['model_predicted_percentage'] = multiple_predictions['prediction_probability'].apply(lambda x: str(x).replace("dict_values([","").replace("])",""))
    results_data = multiple_predictions[['input_data','model_predicted_category','model_predicted_percentage']]
    results_data.to_csv(output_data, index=False)
    return

In [4]:
input_data_path = "DATA/test_data.csv"
output_data_path = "predictions.csv"

In [5]:
run_pipeline = run_pipeline(input_data_path, output_data_path)